#### Лукьянов Кирилл ИУ5-63Б
Вариант 13

## 1. Предобработка данных

In [1]:
import pandas as pd

# Загрузка данных
data = pd.read_csv('dc-wikia-data.csv')

# Проверка первых строк
print(data.head())

# Проверка информации о данных
print(data.info())


   page_id                         name                              urlslug  \
0     1422         Batman (Bruce Wayne)         \/wiki\/Batman_(Bruce_Wayne)   
1    23387        Superman (Clark Kent)        \/wiki\/Superman_(Clark_Kent)   
2     1458   Green Lantern (Hal Jordan)   \/wiki\/Green_Lantern_(Hal_Jordan)   
3     1659     James Gordon (New Earth)     \/wiki\/James_Gordon_(New_Earth)   
4     1576  Richard Grayson (New Earth)  \/wiki\/Richard_Grayson_(New_Earth)   

                ID            ALIGN         EYE        HAIR              SEX  \
0  Secret Identity  Good Characters   Blue Eyes  Black Hair  Male Characters   
1  Secret Identity  Good Characters   Blue Eyes  Black Hair  Male Characters   
2  Secret Identity  Good Characters  Brown Eyes  Brown Hair  Male Characters   
3  Public Identity  Good Characters  Brown Eyes  White Hair  Male Characters   
4  Secret Identity  Good Characters   Blue Eyes  Black Hair  Male Characters   

   GSM              ALIVE  APPEARANCES

In [2]:
# Удаление дубликатов столбцов
data = data.loc[:,~data.columns.duplicated()]

# Проверка после удаления дубликатов
print(data.head())

   page_id                         name                              urlslug  \
0     1422         Batman (Bruce Wayne)         \/wiki\/Batman_(Bruce_Wayne)   
1    23387        Superman (Clark Kent)        \/wiki\/Superman_(Clark_Kent)   
2     1458   Green Lantern (Hal Jordan)   \/wiki\/Green_Lantern_(Hal_Jordan)   
3     1659     James Gordon (New Earth)     \/wiki\/James_Gordon_(New_Earth)   
4     1576  Richard Grayson (New Earth)  \/wiki\/Richard_Grayson_(New_Earth)   

                ID            ALIGN         EYE        HAIR              SEX  \
0  Secret Identity  Good Characters   Blue Eyes  Black Hair  Male Characters   
1  Secret Identity  Good Characters   Blue Eyes  Black Hair  Male Characters   
2  Secret Identity  Good Characters  Brown Eyes  Brown Hair  Male Characters   
3  Public Identity  Good Characters  Brown Eyes  White Hair  Male Characters   
4  Secret Identity  Good Characters   Blue Eyes  Black Hair  Male Characters   

   GSM              ALIVE  APPEARANCES

In [3]:
# Заполнение пропущенных значений
data = data.fillna(method='ffill').fillna(method='bfill')  # Простой способ, можно уточнить в зависимости от данных

# Проверка пропущенных значений
print(data.isnull().sum())

page_id             0
name                0
urlslug             0
ID                  0
ALIGN               0
EYE                 0
HAIR                0
SEX                 0
GSM                 0
ALIVE               0
APPEARANCES         0
FIRST APPEARANCE    0
YEAR                0
dtype: int64


Кодирование категориальных признаков
Преобразуем категориальные признаки в числовые.

In [10]:
from sklearn.preprocessing import LabelEncoder

# Применение LabelEncoder к категориальным признакам
label_encoders = {}
for column in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Проверка преобразованных данных
print(data.head())

   page_id  name  urlslug  ID  ALIGN  EYE  HAIR  SEX  GSM  ALIVE  APPEARANCES  \
0     1422   597      597   2      1    3     0    2    0      1       3093.0   
1    23387  6007     6006   2      1    3     0    2    0      1       2496.0   
2     1458  2487     2487   2      1    4     3    2    0      1       1565.0   
3     1659  3002     3003   1      1    4    16    2    0      1       1316.0   
4     1576  5280     5279   2      1    3     0    2    0      1       1237.0   

   FIRST APPEARANCE    YEAR  
0                14  1939.0  
1               454  1986.0  
2               155  1959.0  
3               460  1987.0  
4                19  1940.0  


## 2. Разделение данных на обучающую и тестовую выборки
Разделим данные на признаки (X) и целевую переменную (y), а затем на обучающую и тестовую выборки.

In [11]:
from sklearn.model_selection import train_test_split

# Предположим, что последний столбец — это целевая переменная
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Проверка размеров выборок
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(5516, 12) (1380, 12) (5516,) (1380,)


## 3. Построение моделей

#### Дерево решений

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score

# Создание и обучение модели
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred_dt = dt_model.predict(X_test)

# Оценка качества модели
accuracy_dt = accuracy_score(y_test, y_pred_dt)
f1_dt = f1_score(y_test, y_pred_dt, average='weighted')

print(f"Decision Tree Accuracy: {accuracy_dt}")
print(f"Decision Tree F1 Score: {f1_dt}")

Decision Tree Accuracy: 0.9971014492753624
Decision Tree F1 Score: 0.996824769433465


#### Случайный лес

In [13]:
from sklearn.ensemble import RandomForestClassifier

# Создание и обучение модели
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred_rf = rf_model.predict(X_test)

# Оценка качества модели
accuracy_rf = accuracy_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')

print(f"Random Forest Accuracy: {accuracy_rf}")
print(f"Random Forest F1 Score: {f1_rf}")

Random Forest Accuracy: 0.7195652173913043
Random Forest F1 Score: 0.7050797217924557


## 4. Оценка качества моделей
Мы используем accuracy (точность) и F1 score (F1-мера) для оценки качества моделей:

- Точность: Доля правильно предсказанных наблюдений к общему числу наблюдений.
- F1-мера: Среднее гармоническое точности и полноты. Полезно, когда важен баланс между этими метриками.

### Выводы
#### Дерево решений:
- Accuracy: указывает на долю верных предсказаний.
- F1 Score: полезно для оценки, когда классы несбалансированы.
    
#### Случайный лес:
Обычно показывает лучшую производительность за счет уменьшения переобучения.
Если F1-мера выше у одной модели по сравнению с другой, это указывает на лучшее качество классификации, особенно при несбалансированных данных.

Таким образом, вы сможете выбрать модель, которая показывает наилучшие результаты по данным метрикам на вашем наборе данных.